## Bus Arrival Time Prediction and Real Time Updating 
##### Group 27

##### Dwell time Dataset Preperation

In [1]:
import pandas as pd


In [3]:
dataset = pd.read_csv('../../data/bus_stop_times_feature_added_all.csv')
poi_dir1 = pd.read_csv('../../data/dwell_times_poi_Kandy-Digana.csv')
poi_dir2 = pd.read_csv('../../data/dwell_times_poi_Digana-Kandy.csv')



In [4]:
dataset.head()

,trip_id,deviceid,direction,bus_stop,date,arrival_time,departure_time,dwell_time,dwell_time_in_seconds_old,day_of_week,...,hour_of_day,day,month,temp,precip,windspeed,conditions,rt(n-1),stop_type,dwell_time_in_seconds
0,1,262,1,101,2021-10-01,06:40:58,06:42:12,0:01:14,74.0,4,...,6,1,10,20.0,0.0,6.1,Partially cloudy,69.0,pro,74.0
1,1,262,1,102,2021-10-01,06:45:42,06:45:42,0:00:00,0.0,4,...,6,1,10,20.0,0.0,6.1,Partially cloudy,210.0,mod,0.0
2,1,262,1,103,2021-10-01,06:53:58,06:54:04,0:00:06,6.0,4,...,6,1,10,20.0,0.0,6.1,Partially cloudy,496.0,mod,6.0
3,1,262,1,104,2021-10-01,06:57:19,06:57:19,0:00:00,0.0,4,...,6,1,10,20.0,0.0,6.1,Partially cloudy,195.0,br,0.0
4,1,262,1,105,2021-10-01,06:58:56,07:02:27,0:03:31,211.0,4,...,6,1,10,20.0,0.0,6.1,Partially cloudy,97.0,pro,211.0


In [5]:
poi_dir1['direction'] = 1.0
poi_dir2['direction'] = 2.0

In [6]:
poi_features = ['stop_id','direction','place_of_worship','place_of_education', 'hospital', 
'tourist_attractions', 'commercial_places', 'public_places','nearby_bus_stops','total_poi_count', 'average_poi_distance']

In [7]:
#concatanate poi datasets along columns
poi_dir1 = poi_dir1[poi_features]
poi_dir2 = poi_dir2[poi_features]
poi_data = pd.concat([poi_dir1,poi_dir2])
poi_data.shape

(31, 11)

In [8]:
poi_data.rename(columns={'stop_id':'bus_stop'},inplace=True)
poi_data.reset_index(drop=True,inplace=True)


In [9]:
poi_data

,bus_stop,direction,place_of_worship,place_of_education,hospital,tourist_attractions,commercial_places,public_places,nearby_bus_stops,total_poi_count,average_poi_distance
0,BT01,1.0,8,2,0,39,3,11,4,15,0.002018
1,101,1.0,3,1,0,23,1,7,1,13,0.002210
2,102,1.0,0,8,0,2,0,1,0,6,0.002406
3,103,1.0,4,0,0,2,0,1,0,5,0.002081
4,104,1.0,2,0,0,1,0,1,0,3,0.001165
5,105,1.0,2,1,0,0,0,2,0,3,0.001913
6,106,1.0,0,0,0,3,0,13,0,6,0.000855
7,107,1.0,1,0,0,2,0,6,0,8,0.001087
8,108,1.0,1,0,1,3,0,6,0,11,0.001867
9,109,1.0,0,0,0,4,0,6,0,9,0.000794


In [10]:
poi_data = poi_data.drop(poi_data[poi_data['bus_stop'] =='BT01'].index)

In [11]:
poi_data = poi_data.drop(poi_data[poi_data['bus_stop'] =='BT02'].index)

In [12]:
poi_data['bus_stop'] = poi_data['bus_stop'].astype('int64')

In [13]:
dataset_poi = pd.merge(left=dataset,right=poi_data,how='left',on=['bus_stop','direction'])
dataset_poi = dataset_poi.dropna()

In [14]:
dataset_poi.head()

,trip_id,deviceid,direction,bus_stop,date,arrival_time,departure_time,dwell_time,dwell_time_in_seconds_old,day_of_week,...,dwell_time_in_seconds,place_of_worship,place_of_education,hospital,tourist_attractions,commercial_places,public_places,nearby_bus_stops,total_poi_count,average_poi_distance
0,1,262,1,101,2021-10-01,06:40:58,06:42:12,0:01:14,74.0,4,...,74.0,3,1,0,23,1,7,1,13,0.002210
1,1,262,1,102,2021-10-01,06:45:42,06:45:42,0:00:00,0.0,4,...,0.0,0,8,0,2,0,1,0,6,0.002406
2,1,262,1,103,2021-10-01,06:53:58,06:54:04,0:00:06,6.0,4,...,6.0,4,0,0,2,0,1,0,5,0.002081
3,1,262,1,104,2021-10-01,06:57:19,06:57:19,0:00:00,0.0,4,...,0.0,2,0,0,1,0,1,0,3,0.001165
4,1,262,1,105,2021-10-01,06:58:56,07:02:27,0:03:31,211.0,4,...,211.0,2,1,0,0,0,2,0,3,0.001913


In [15]:
print(dataset_poi.shape)

(91683, 42)


In [17]:
dataset_poi.to_csv('../../data/bus_stop_times_poi_data_added_modified.csv')